In [25]:
import sys
import os

PROJECT_DIR = '/home/evgenshuben/Desktop/gitReps/YandexCup/'
sys.path.append(PROJECT_DIR)
os.chdir(PROJECT_DIR)

In [157]:
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra import initialize, compose, initialize_config_dir
from hydra.utils import instantiate

import torch.nn as nn
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
from tqdm import tqdm
from functools import reduce

In [192]:
%load_ext autoreload
%autoreload 2

from scr.train import Trainer
from scr.utils import (reduce_func, save_test_predictions)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# info

Тут попробуем сделать стекинг нескольких моделей предобученных

Пробую взять вектора от моделей на тесте и усреднить их просто

Написал функцию для сохранения тестовых эмбеддингов

# Load config

In [ ]:
# config_dir = '/home/evgenshuben/Desktop/gitReps/YandexCup/configs'
# config_name = 'config'


# with initialize_config_dir(config_dir=config_dir, version_base=None):
#     cfg = compose(config_name=config_name)
#     # cfg['dataset']['train']['train']['dataset_path']

In [4]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-2/hydra/.hydra/config.yaml"



cfg = OmegaConf.load(config_path)
cfg.enviroment.device = 'cpu'
cfg.model

{'_target_': 'scr.models.cnn.CNN1d', 'emb_size': 512, 'num_classes': '${num_classes}', 'input_norm': True}

In [5]:
trainer = Trainer(cfg)

### embeddings calculate

Насчитаем эмбеддинги на каждой сильной модели

###### 6

In [28]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-6/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
cfg.model

{'_target_': 'scr.models.backbone.BackBone', 'backbone_name': 'hgnetv2_b4.ssld_stage2_ft_in1k', 'emb_size': 512, 'num_classes': '${num_classes}'}

In [29]:
trainer = Trainer(cfg)

/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

In [30]:
%%time
trainer.test_embeddings()

100%|██████████| 55170/55170 [05:04<00:00, 180.94it/s]


CPU times: user 5min 4s, sys: 24.1 s, total: 5min 28s
Wall time: 5min 5s


###### 4

In [31]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-4/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
cfg.model

{'_target_': 'scr.models.backbone.BackBone', 'backbone_name': 'tf_efficientnet_b4.ns_jft_in1k', 'emb_size': 512, 'num_classes': '${num_classes}'}

In [32]:
trainer = Trainer(cfg)

/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

In [33]:
%%time
trainer.test_embeddings()

100%|██████████| 55170/55170 [09:02<00:00, 101.67it/s]


CPU times: user 9min 3s, sys: 26.7 s, total: 9min 30s
Wall time: 9min 3s


###### 14

In [148]:
# Путь к конфигурационному файлу, сохраненному Hydra
config_path = "/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-14/hydra/.hydra/config.yaml"
cfg = OmegaConf.load(config_path)
# cfg.enviroment.device = 'cpu'
cfg.model

{'_target_': 'scr.models.backbone.BackBone', 'backbone_name': 'regnety_040.ra3_in1k', 'emb_size': 512, 'num_classes': '${num_classes}'}

In [149]:
trainer = Trainer(cfg)

/home/evgenshuben/Desktop/gitReps/YandexCup/scr/train.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(self.cfg.pipeline.model_ckpt

In [150]:
%%time
trainer.test_embeddings()

100%|██████████| 55170/55170 [07:08<00:00, 128.82it/s]


CPU times: user 7min 8s, sys: 26.4 s, total: 7min 34s
Wall time: 7min 8s


### Embeddings pooling

Тут попробую усреднить эмбеды от предобученных моделей
с нормировкой эмбедов

In [151]:
emb_6_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-6/data/test_embeddings.pq')
emb_4_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-4/data/test_embeddings.pq')
emb_14_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-14/data/test_embeddings.pq')

In [152]:
from sklearn.preprocessing import normalize

def normalize_embeddings(embedding):
    return embedding / np.linalg.norm(embedding)

In [42]:
emb_6_pd.head()

,embedding
trackids,
1,"[-0.07723766, -0.612644, -0.72078836, 0.334147..."
7159,"[-1.4252408, 0.33825552, 0.2531855, 1.2647257,..."
13557,"[-0.6442174, -0.65833646, 2.0886574, 0.1040712..."
14915,"[-0.023006344, -0.38933963, 0.6891218, -0.0324..."
16415,"[-0.22624709, -0.20559229, 0.3041806, -1.25063..."


In [43]:
emb_4_pd.head()

,embedding
trackids,
1,"[1.7688749, 1.8406674, 0.41593462, -0.7498508,..."
7159,"[-0.052356258, -0.15832356, 0.67157507, -0.228..."
13557,"[2.2554877, -1.0612838, 1.0043125, -3.5117183,..."
14915,"[0.5620366, 0.10097464, 0.48317677, -0.4449085..."
16415,"[0.55709475, -0.7496168, 0.22376603, 0.2634193..."


In [153]:
emb_6_pd['embedding'] = emb_6_pd['embedding'].apply(normalize_embeddings)
emb_4_pd['embedding'] = emb_4_pd['embedding'].apply(normalize_embeddings)
emb_14_pd['embedding'] = emb_14_pd['embedding'].apply(normalize_embeddings)

In [45]:
emb_6_pd.head()

,embedding
trackids,
1,"[-0.0030853231, -0.02447258, -0.028792497, 0.0..."
7159,"[-0.06812118, 0.01616735, 0.012101321, 0.06044..."
13557,"[-0.026385976, -0.02696427, 0.085547626, 0.004..."
14915,"[-0.0010185345, -0.017236805, 0.030508729, -0...."
16415,"[-0.0097744325, -0.008882094, 0.013141353, -0...."


In [46]:
emb_4_pd.head()

,embedding
trackids,
1,"[0.08058438, 0.08385501, 0.018948672, -0.03416..."
7159,"[-0.0025522916, -0.0077180434, 0.03273831, -0...."
13557,"[0.08772994, -0.041279927, 0.03906396, -0.1365..."
14915,"[0.02298532, 0.004129508, 0.019760232, -0.0181..."
16415,"[0.024934666, -0.033551645, 0.010015408, 0.011..."


In [198]:
def merge_dfs(df1, df2, suffix1, suffix2):
    return df1.join(df2, how='inner', lsuffix=suffix1, rsuffix=suffix2)

In [199]:
dataframes = [emb_6_pd, emb_4_pd, emb_14_pd] 
merged_df = dataframes[0]

suffixes = [f'_{i}' for i in range(len(dataframes))]

for i in range(1, len(dataframes)):
    merged_df = merged_df.join(dataframes[i], how='inner', lsuffix=suffixes[i-1], rsuffix=suffixes[i])

merged_df.rename(columns={'embedding': f'embedding_{len(dataframes)}'}, inplace=True)

In [200]:
merged_df.head()

,embedding_0,embedding_1,embedding_3
trackids,,,
1,"[-0.0030853231, -0.02447258, -0.028792497, 0.0...","[0.08058438, 0.08385501, 0.018948672, -0.03416...","[-0.0141357435, -0.022207366, 0.047852542, 0.0..."
7159,"[-0.06812118, 0.01616735, 0.012101321, 0.06044...","[-0.0025522916, -0.0077180434, 0.03273831, -0....","[-0.002881172, 0.0020705282, -0.0011530791, -0..."
13557,"[-0.026385976, -0.02696427, 0.085547626, 0.004...","[0.08772994, -0.041279927, 0.03906396, -0.1365...","[-0.02878957, 0.008982886, -0.011043699, 0.008..."
14915,"[-0.0010185345, -0.017236805, 0.030508729, -0....","[0.02298532, 0.004129508, 0.019760232, -0.0181...","[0.05659094, -0.062711574, 0.011440689, -0.070..."
16415,"[-0.0097744325, -0.008882094, 0.013141353, -0....","[0.024934666, -0.033551645, 0.010015408, 0.011...","[-0.036691837, 0.00088549685, 0.0044573215, 0...."


In [201]:
def average_embeddings(merged_df, weights=None):
    # Находим все столбцы, имена которых начинаются с 'embedding_'
    embedding_columns = [col for col in merged_df.columns if col.startswith('embedding_')]
    
    # Убедимся, что есть хотя бы один столбец для усреднения
    if not embedding_columns:
        raise ValueError("Нет столбцов с эмбеддингами для усреднения.")
    
    # Если веса не переданы, используем равные веса
    if weights is None:
        weights = np.ones(len(embedding_columns))
    elif len(weights) != len(embedding_columns):
        raise ValueError("Длина списка весов должна совпадать с количеством эмбеддингов.")

    # Применяем взвешенное усреднение по найденным столбцам
    merged_df['embedding'] = merged_df[embedding_columns].apply(
        lambda row: np.average(row, weights=weights), axis=1
    )

In [209]:
(-0.0030853231 + 0.08058438 -0.0141357435*0.7)/3

0.022534678816666667

In [222]:
average_embeddings(merged_df, weights=[1.0, 1.0, 0.7])

In [223]:
merged_df.head()

,embedding_0,embedding_1,embedding_3,embedding
trackids,,,,
1,"[-0.0030853231, -0.02447258, -0.028792497, 0.0...","[0.08058438, 0.08385501, 0.018948672, -0.03416...","[-0.0141357435, -0.022207366, 0.047852542, 0.0...","[0.025038531, 0.016236028, 0.008760354, -0.005..."
7159,"[-0.06812118, 0.01616735, 0.012101321, 0.06044...","[-0.0025522916, -0.0077180434, 0.03273831, -0....","[-0.002881172, 0.0020705282, -0.0011530791, -0...","[-0.02692233, 0.0036661765, 0.016308323, 0.017..."
13557,"[-0.026385976, -0.02696427, 0.085547626, 0.004...","[0.08772994, -0.041279927, 0.03906396, -0.1365...","[-0.02878957, 0.008982886, -0.011043699, 0.008...","[0.015256022, -0.02294673, 0.04328926, -0.0468..."
14915,"[-0.0010185345, -0.017236805, 0.030508729, -0....","[0.02298532, 0.004129508, 0.019760232, -0.0181...","[0.05659094, -0.062711574, 0.011440689, -0.070...","[0.02280757, -0.02111311, 0.021584237, -0.0254..."
16415,"[-0.0097744325, -0.008882094, 0.013141353, -0....","[0.024934666, -0.033551645, 0.010015408, 0.011...","[-0.036691837, 0.00088549685, 0.0044573215, 0....","[-0.003897797, -0.015486626, 0.009732179, -0.0..."


In [224]:
from sklearn.metrics import pairwise_distances_chunked

In [225]:
embeddings = merged_df['embedding'].tolist()
trackids = merged_df.index.tolist()

In [226]:
predictions = []
for chunk_result in tqdm(pairwise_distances_chunked(
        embeddings, metric='cosine', reduce_func=reduce_func, working_memory=100
)):
    for query_indx, query_nearest_items in chunk_result:
        predictions.append((trackids[query_indx], [trackids[nn_indx] for nn_indx in query_nearest_items]))

233it [03:07,  1.25it/s]


In [227]:
save_test_predictions(
    predictions, output_dir='outputs_test', experiment_number='stacking_mean_normalized_4_6_14'
)

In [47]:
merged_df = emb_6_pd.join(emb_4_pd, how='inner', lsuffix='_6', rsuffix='_4')

In [48]:
merged_df['embedding'] = merged_df.apply(
    lambda row: np.mean([row['embedding_6'], row['embedding_4']], axis=0),
    axis=1
)

In [184]:
merged_df.head()

,embedding_0,embedding_1,embedding_3
trackids,,,
1,"[-0.0030853231, -0.02447258, -0.028792497, 0.0...","[0.08058438, 0.08385501, 0.018948672, -0.03416...","[-0.0141357435, -0.022207366, 0.047852542, 0.0..."
7159,"[-0.06812118, 0.01616735, 0.012101321, 0.06044...","[-0.0025522916, -0.0077180434, 0.03273831, -0....","[-0.002881172, 0.0020705282, -0.0011530791, -0..."
13557,"[-0.026385976, -0.02696427, 0.085547626, 0.004...","[0.08772994, -0.041279927, 0.03906396, -0.1365...","[-0.02878957, 0.008982886, -0.011043699, 0.008..."
14915,"[-0.0010185345, -0.017236805, 0.030508729, -0....","[0.02298532, 0.004129508, 0.019760232, -0.0181...","[0.05659094, -0.062711574, 0.011440689, -0.070..."
16415,"[-0.0097744325, -0.008882094, 0.013141353, -0....","[0.024934666, -0.033551645, 0.010015408, 0.011...","[-0.036691837, 0.00088549685, 0.0044573215, 0...."


In [50]:
from sklearn.metrics import pairwise_distances_chunked

повторим пайплайн из трейнера по сборке ближайших соседей

In [60]:
embeddings = merged_df['embedding'].tolist()
trackids = merged_df.index.tolist()

In [63]:
predictions = []
for chunk_result in pairwise_distances_chunked(
        embeddings, metric='cosine', reduce_func=reduce_func, working_memory=100
):
    for query_indx, query_nearest_items in chunk_result:
        predictions.append((trackids[query_indx], [trackids[nn_indx] for nn_indx in query_nearest_items]))

In [65]:
trainer.cfg.pipeline.test_output_dir

'outputs_test'

In [72]:
save_test_predictions(
    predictions, output_dir='outputs_test', experiment_number='stacking_mean_46'
)

###### стакинг без нормировки

Сработал хуже

In [73]:
emb_6_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-6/data/test_embeddings.pq')
emb_4_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-4/data/test_embeddings.pq')

In [75]:
merged_df = emb_6_pd.join(emb_4_pd, how='inner', lsuffix='_6', rsuffix='_4')

merged_df['embedding'] = merged_df.apply(
    lambda row: np.mean([row['embedding_6'], row['embedding_4']], axis=0),
    axis=1
)

In [76]:
embeddings = merged_df['embedding'].tolist()
trackids = merged_df.index.tolist()

In [79]:
predictions = []
for chunk_result in tqdm(pairwise_distances_chunked(
        embeddings, metric='cosine', reduce_func=reduce_func, working_memory=100
)):
    for query_indx, query_nearest_items in chunk_result:
        predictions.append((trackids[query_indx], [trackids[nn_indx] for nn_indx in query_nearest_items]))

233it [03:05,  1.25it/s]


In [141]:
# predictions[0]

In [80]:
save_test_predictions(
    predictions, output_dir='outputs_test', experiment_number='stacking_mean_without_normalize_46'
)

###### Скоры от моделей

Теперь попробуем сложить не эмбеды а скоры от моделей

для этого надо изменить редьюсер, чтобы он возвращал еще и скор

In [102]:
def score_reduce_func(D_chunk, start, top_size=100):
    # Находим индексы ближайших соседей
    nearest_items = np.argsort(D_chunk, axis=1)[:, :top_size + 1]
    
    # Извлекаем соответствующие косинусные расстояния
    nearest_distances = np.take_along_axis(D_chunk, nearest_items, axis=1)
    
    # Возвращаем пары (индексы и близости)
    return [
        (i, items[items != i], 1 - nearest_distances[idx][items != i])  # 1 - расстояние для получения близости
        for idx, (i, items) in enumerate(zip(range(start, start + D_chunk.shape[0]), nearest_items))
    ]

In [99]:
emb_6_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-6/data/test_embeddings.pq')
embeddings = emb_6_pd['embedding'].tolist()
trackids = emb_6_pd.index.tolist()

In [105]:
predictions = []
for chunk_result in tqdm(pairwise_distances_chunked(
        embeddings, metric='cosine', reduce_func=score_reduce_func, working_memory=100
)):
    for query_indx, query_nearest_items, query_similarities in chunk_result:
        # Добавляем результаты: (ID трека запроса, [ID соседей], [косинусные близости соседей])
        predictions.append(
            (trackids[query_indx],
             [trackids[nn_indx] for nn_indx in query_nearest_items],
             query_similarities.tolist())  # Преобразуем близости в список
        )

233it [03:04,  1.26it/s]


In [113]:
result6_dict = {}

# Проходимся по списку predictions и заполняем словарь
for track_id, candidate_ids, similarities in predictions:
    # Создаем пары (candidate_id, similarity) и добавляем в словарь
    result6_dict[track_id] = list(zip(candidate_ids, similarities))


In [116]:
result6_dict[1][:5]

[(53098, 0.44250911474227905),
 (44946, 0.4099239706993103),
 (18000, 0.40310966968536377),
 (7711, 0.3896787166595459),
 (11825, 0.38576972484588623)]

In [107]:
emb_4_pd = pd.read_parquet('/home/evgenshuben/Desktop/gitReps/YandexCup/outputs_val/run-4/data/test_embeddings.pq')
embeddings = emb_4_pd['embedding'].tolist()
trackids = emb_4_pd.index.tolist()

In [108]:
predictions_4 = []
for chunk_result in tqdm(pairwise_distances_chunked(
        embeddings, metric='cosine', reduce_func=score_reduce_func, working_memory=100
)):
    for query_indx, query_nearest_items, query_similarities in chunk_result:
        # Добавляем результаты: (ID трека запроса, [ID соседей], [косинусные близости соседей])
        predictions_4.append(
            (trackids[query_indx],
             [trackids[nn_indx] for nn_indx in query_nearest_items],
             query_similarities.tolist())  # Преобразуем близости в список
        )

233it [03:05,  1.26it/s]


In [111]:
result4_dict = {}

# Проходимся по списку predictions и заполняем словарь
for track_id, candidate_ids, similarities in predictions_4:
    # Создаем пары (candidate_id, similarity) и добавляем в словарь
    result4_dict[track_id] = list(zip(candidate_ids, similarities))


In [117]:
result4_dict[1][:5]

[(27655, 0.38579368591308594),
 (23797, 0.3703147768974304),
 (23017, 0.36620032787323),
 (44929, 0.335546612739563),
 (5910, 0.3270080089569092)]

In [118]:
from collections import defaultdict

In [119]:
combined_scores = defaultdict(lambda: defaultdict(list))

# Добавляем данные из первого словаря
for track_id, candidates in result4_dict.items():
    for candidate_id, similarity in candidates:
        combined_scores[track_id][candidate_id].append(similarity)

# Добавляем данные из второго словаря
for track_id, candidates in result6_dict.items():
    for candidate_id, similarity in candidates:
        combined_scores[track_id][candidate_id].append(similarity)

In [127]:
# Усредняем скоры и формируем окончательный словарь
result_dict = {}
for track_id, candidate_scores in combined_scores.items():
    averaged_candidates = [
        (candidate_id, sum(scores) / len(scores))  # Усредняем скоры
        for candidate_id, scores in candidate_scores.items()
    ]
    # Сортируем кандидатов по убыванию усредненного similarity
    averaged_candidates.sort(key=lambda x: x[1], reverse=True)
    result_dict[track_id] = averaged_candidates

# Пример вывода первых нескольких элементов
# print(list(result_dict.items())[:5])

In [140]:
result_dict[1]

[(53098, 0.44250911474227905),
 (44946, 0.4099239706993103),
 (18000, 0.40310966968536377),
 (7711, 0.3896787166595459),
 (27655, 0.38579368591308594),
 (11825, 0.38576972484588623),
 (23797, 0.3703147768974304),
 (47143, 0.36950981616973877),
 (12057, 0.3681948184967041),
 (23017, 0.36620032787323),
 (43451, 0.3638312816619873),
 (7309, 0.36325711011886597),
 (14646, 0.3602888584136963),
 (40312, 0.35778361558914185),
 (54764, 0.35691606998443604),
 (5922, 0.35435497760772705),
 (31379, 0.3540458679199219),
 (6765, 0.35104990005493164),
 (45792, 0.3501570224761963),
 (52400, 0.3492717742919922),
 (45217, 0.348804235458374),
 (37628, 0.3467860221862793),
 (36329, 0.3449831008911133),
 (12409, 0.34446918964385986),
 (48235, 0.343350350856781),
 (41785, 0.3378169536590576),
 (3934, 0.33765870332717896),
 (50139, 0.33763760328292847),
 (8872, 0.3368837833404541),
 (36702, 0.33583152294158936),
 (42406, 0.3357158899307251),
 (44929, 0.335546612739563),
 (16472, 0.3354657292366028),
 (49456

In [142]:
predictions_combined = [
    (track_id, [candidate_id for candidate_id, _ in candidates[:100]])
    for track_id, candidates in result_dict.items()
]

In [145]:
save_test_predictions(
    predictions_combined, output_dir='outputs_test', experiment_number='stacking_models_score_46'
)